In [7]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from other_repos.pyDMS.pyDMS.pyDMS import *
import time

os.environ["GDAL_MAX_DATASET_POOL_SIZE"] = "600"
# get tiles for Brandenburg
bran = pd.read_csv('/data/Aldhani/eoagritwin/misc/state_tile_csv/clipped_grid_bran_tiles.csv')

# create vrts of slope and aspect
path_to_slope = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/SLOPE/'
path_to_aspect = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/ASPECT/'
temp_dump = '/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump/'


# get all highRes datasets: S2 composites, aspect, ratio and incedence for the tile/time and stack them (in a composite if more than one tile is processed)
# set the highRes S2 tiles that will be processed
tiles_to_process = createFORCEtileLIST(list(bran['Tile_X']),
                                        list(bran['Tile_Y']))


# tiles_to_process = get_forceTSI_output_Tiles(getFilelist(path_to_slope, '.tif'))

# slope-tiles
slopes = [file for file in getFilelist(path_to_slope, '.tif') if any(tile in file for tile in tiles_to_process)] # if any tile name is in file
# aspect-tiles
aspects = [file for file in getFilelist(path_to_aspect, '.tif') if any(tile in file for tile in tiles_to_process)] # if any tile name is in file

# get those tiles (and composite if more than one tile is provided)
if len(tiles_to_process) == 1:
    
    slope_path = slopes[0]
    aspect_path = aspects[0]

else:
    slope_path = f'{temp_dump}SLOPE.vrt'
    gdal.BuildVRT(slope_path, slopes)

    aspect_path = f'{temp_dump}ASPECT.vrt'
    gdal.BuildVRT(aspect_path, aspects)


In [15]:
# year 
year = 2019#range(2017,2025):
  
# paths
path_to_S2_tiles = f'/data/Aldhani/eoagritwin/force/output/Guzinski/{year}/'


##### which tiles should be processed
# get a list with all available tiles
files = getFilelist(f'{path_to_S2_tiles}/tiles', '.tif', deep=True) 
files = [file for file in files if any(tile in file for tile in tiles_to_process)]
date_list = check_forceTSI_compositionDates(files)


#### S2 composites are time sensitive (need to be aligned with date of LST observation), so is incidence

for date in date_list:
# date = date_list[0] # will be replaced through loop
    if date != '20190705':
        continue
    # get those tiles (and composite if more than one tile is provided)
    if len(tiles_to_process) == 1:

        tilesS2 = [file for file in getFilelist(path_to_S2_tiles, '.tif', deep=True) if tiles_to_process[0] in file and f'{date}.tif' in file]
        S2_path = f'{temp_dump}S2_{date}.vrt'
        gdal.BuildVRT(S2_path, tilesS2)

    else:
        tilesS2 = [file for file in getFilelist(path_to_S2_tiles, '.tif', deep=True) if any(tile in file for tile in tiles_to_process) and f'{date}.tif' in file] 
        force_to_vrt(tilesS2,
                getCOLORSinOrderFORCELIST(tilesS2, list(dict.fromkeys(tile.split('SEN2L_')[-1].split('_TSI')[0] for tile in tilesS2)), single=False),
                f'{temp_dump}S2_{date}',
                False,
                bandnames= list(dict.fromkeys(tile.split('SEN2L_')[-1].split('_TSI')[0] for tile in tilesS2)))
        print(list(dict.fromkeys(tile.split('SEN2L_')[-1].split('_TSI')[0] for tile in tilesS2)))
        S2_path = [file for file in getFilelist(f'{temp_dump}S2_{date}', '.vrt', deep=True) if '_Cube' in file][0]
        
        # determine LST and incidence files associated with respective S2 composite
    band_dict = transform_compositeDate_into_LSTbands(date, 4)


    # stat used for compositing
    for comp_stat in ['minVZA', 'maxLST']:
        path_to_incident = f'/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/INCIDENCE/{comp_stat}/{year}/'
        path_to_LST = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/{comp_stat}/{year}/'

        # get all LST bands that can be sharped with the S2 composite at this date (and sun angle incidence files as well, as they are dependent on that date
        incid_dates = []
        LSTs = []

        for k, v in band_dict.items():
            month = v['month']
            band = int(v['band'])
            v_path = f'{path_to_LST}Daily_LST_{comp_stat}_{year}_{month}.tif'
            ds = gdal.Open(v_path, 0)
            LST_arr = ds.GetRasterBand(band).ReadAsArray() # store as single Tiff in temp
            makeTif_np_to_matching_tif(LST_arr, v_path, f'{temp_dump}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif')

            # store the paths for selecting incidence for corresponding LST
            incid_dates.append(f'{year}_{month}_{band:02d}.tif')
            LSTs.append(f'{temp_dump}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif')
            ##### loop over the LST files and go

        for idx, LST_file in enumerate(LSTs):
            # incidence-tiles
            incids = [file for file in getFilelist(path_to_incident, '.tif', deep=True) if any(tile in file for tile in tiles_to_process) and incid_dates[idx] in file] 
            
            # get those tiles (and composite if more than one tile is provided)
            if len(tiles_to_process) == 1:
                incid_path = incids[0]

            else:
                incid_path = f'{temp_dump}INCIDENCE.vrt'
                gdal.BuildVRT(incid_path, incids)

            # sanity check for incidence and LST date
            if (LSTs[idx].split(f'{year}')[-1] == incids[0].split(f'{year}')[-1]):
                
                # get LST file
                lowRes_path = LSTs[idx]
                # create highRes file through exapnding the vrt of S2
                highRes_path = f'{temp_dump}HIGHRES.vrt'
                gdal.BuildVRT(highRes_path, [S2_path, slope_path, aspect_path, incid_path], separate=True)
            else:
                raise ValueError('Something is seriously wrong with the alignment of LST and incidence dates!!!!!')

all dates of composites are the same :)
Vrt might already exist - please check!!
['BLU', 'GRN', 'NIR', 'RE1', 'RE2', 'RE3', 'RED', 'SW1', 'SW2']


In [5]:
import numpy as np
a = 0
for movWin in range(20,55,5):
    for cv in range(5,40,5):
        for regrat in np.arange(0.1, 0.35, 0.05):
                a += 1
print(a)                           

245
